Search notebook component?



In [ ]:
import { Component, ModuleWithProviders, NgModule } from '@angular/core';
import { COMMON_MODULES } from '../../../imports/core.module';
import { RouterModule, Routes } from '@angular/router';
import { SearchService } from '../../../imports/search.service';

@Component({
    selector: 'bc-search',
    template: `
        <form>
            <md-input-container>
                <input mdInput name="search" required type="text"
                       placeholder="Search"
                       maxlength="100" [(ngModel)]="query" (change)="search()">
            </md-input-container>
        </form>
    `,
    styles: [ `
        md-input-container {
            width: 100%;
        }
    ` ]
})
export class SearchComponent {
    query = '';

    constructor(public service: SearchService) {
    }

    search(): void {
        this.service.search(this.query).subscribe(r => {
            console.log('searched ' + r);
        });
    }
}

export const COMPONENTS = [
    SearchComponent
];

export const authRoutes: Routes = [
    {
        path: '',
        component: SearchComponent,
        data: {roles: [ 'anonymous', 'user' ]}
    }
];
export const routing: ModuleWithProviders = RouterModule.forChild(authRoutes);

@NgModule({
    imports: [
        ...COMMON_MODULES,
        routing
    ],
    declarations: COMPONENTS,
    exports: COMPONENTS
})
export class SearchModule {
}



In [ ]:
Search notebook service?

In [ ]:
import { Observable } from 'rxjs/Observable';
import { Http } from '@angular/http';
import { Injectable } from '@angular/core';
import { Response } from '@angular/http';

export let callbackUrl = 'localhost';

@Injectable()
export class SearchService {
    constructor(public http: Http) {
    }

    search(query: string): Observable<Response> {
        console.log('Searching ' + query);
        return this.http.post(callbackUrl, {query});
    }
}


Display code results?

In [ ]:
import { SearchService } from '../../../imports/search.service';
import { ChangeDetectorRef, Component, OnDestroy, OnInit } from '@angular/core';
import { Subscription } from 'rxjs/Subscription';
import * as Prism from 'prismjs';

@Component({
    selector: 'bc-results',
    template: `
        <pre [class]="'language-'+(r.lang||'javascript')" *ngFor="let r of results" [innerHTML]="highlight(r.code)">
        </pre>
    `,
    styleUrls: [ './results.component.scss' ]
})
export class ResultsComponent implements OnInit, OnDestroy {
    query = '';
    results: Array<any> = [];
    private resultsSub: Subscription;

    constructor(public service: SearchService,
                public ref: ChangeDetectorRef) {
    }

    ngOnInit(): void {
        this.resultsSub = this.service.results(this.query).subscribe(r => {
            this.results = [ (r as Array<any>)[ 0 ] ];
            this.ref.detectChanges();
        });
    }

    ngOnDestroy(): void {
        this.resultsSub.unsubscribe();
    }

    highlight(code: string): string {
        return Prism.highlight(code, Prism.languages.javascript);
    }
}



Sockify search notebook provider?

In [ ]:
var client = require('socket.io-client').connect('http://localhost:8000');
var importer = require('../Core');

$$.async();
client.emit('handle', 'SearchService');
client.on('resolve', (name, search) => {
    var interpretObject;
    if(name == 'SearchService.prototype.search') {
        client.emit('result', 'SearchService.prototype.search', 'Searching notebooks for ' + search);
        importer.importNotebook('../Core/interpret all notebooks.ipynb')
            .then(nb => {
                var interpret = nb['interpret'];
                interpretObject = nb['interpretObject'];
                return interpret(search);
            })
            .then(r => {
                return interpretObject(r);
            })
            .then(r => {
                client.emit('result', 'SearchService.prototype.results', r);
                $$.done();
            });
    }
});



In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/universal';
var server = require(project + '/src/server/sockify-server.js').sockifyServer(8000);
var client = require('socket.io-client').connect('http://localhost:8000');
client.emit('close');
